In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sbn
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Problem 1 

You are provided with simple data frame with only numeric data in columns.

**Part I**

Standardize (mean 0, variance 1) it by columns.

**Part II**

Standardize (mean 0, variance 1) it by rows.

In [ ]:
np.random.seed(1010)

df = pd.DataFrame(np.random.normal(100, 15, (10, 3)))
df

In [ ]:
#Part I
col_stan = (df - df.mean(0)) / df.std(0)
col_stan

#Part II
row_stan = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
row_stan

## Problem 2 

Center numeric data frame jointly by rows and columns (so both rows and columns should have mean 0).

In [ ]:
col_cen = df - df.mean(0)
all_cen = col_cen.sub(col_cen.mean(1), axis=0)
all_cen

## Problem 3 

In this exercise you will use `tips` dataset. It records tips given by people in a restaurant together with some additional covariates.
Your task is to compute mean and standard deviation for tips broken down by day (week day). Do you see a pattern here?
What is interesting about Saturday?

HINT. Maybe `groupby` will be useful here?

In [ ]:
tips = sbn.load_dataset('tips')

tips

In [ ]:
tips.groupby(['day'])['tip'].agg(['mean', 'std']).rename(columns={'mean': 'tips mean', 'std': 'tips sd'})

## Problem 4

Read in dataset from the `assertiveness.tsv` file. Compute both number and fraction of missing values
per column.

**Part II**

Substitute missing values by mean values (by column).

HINT. Look at the structure of the file before reading it. You may use any simple text editor that you like.

In [ ]:
tsk = pd.read_csv('assertiveness.tsv', sep="\t")

In [ ]:
tsk

In [ ]:
#Part I
def nan_sum(tsk):
  return tsk.isnull().sum()

def nan_frac(tsk):
  return tsk.isnull().sum() / len(tsk.index)

tsk.agg([nan_sum, nan_frac]).rename(index={'nan_sum': 'Number of nan values', 'nan_frac': 'Fraction of nan values'})

In [ ]:
#Part II
tsk.fillna(tsk.mean())

## Problem 5 

In this exercise you will work with `exercises` dataset that stores data about pulse of different subjects after
1, 15 and 30 minutes during different kinds of physical activities. Subjects are also on different diets.

Use this data to determine which combination of diet (`diet`) and kind of physical activity (`kind`) is associated
with the greater increase of pulse between the 1st and 30th minute of training.

HINT. Use `groupby` for this.

In [ ]:
df = sbn.load_dataset('exercise')
df.head()

In [ ]:
df['pulse_change'] = df['pulse'].pct_change(periods=2) #Percentage change between 30th and 1st minute
min_30 = df['time'] == '30 min' #only 30th min rows have a number that describes 30-1 change
result = df[min_30].groupby(['diet', 'kind'])['pulse_change'].mean()
result
print("\nThe greatest increase of pulse between the 1st and 30th minute of training is accomplished by", result.idxmax())

## Problem 6 

Here we will use `exercise` dataset again. Now your task is to illustrate _pulse curves_ for individual subjects.
Your task is to plot curves showing pulse values for individual subjects with respect to time.
Hence, the `x` axis should be `time` variable from the dataset and the `y` axis should be `pulse`.
Moreover, denote `diet` with color and `kind` with line type (i.e. solid line and dashed line).

HINT. When using `matplotlib` you will have to call `plot` method multiple times, but remember that you can control
color with `c` (or `color`) parameter and line type with `linestyle` parameter.

HINT2. Matplotlib [line style reference](https://matplotlib.org/gallery/lines_bars_and_markers/line_styles_reference.html).

In [ ]:
df = sbn.load_dataset('exercise')
df.head()

In [ ]:
sbn.lineplot(x='time', y='pulse', hue='diet', data=df, style='kind', palette=['mediumturquoise', 'crimson'], units='id', estimator=None, ci = None, legend = "brief")

## Problem 7 

Use `exercise` dataset again. 

Create a new column `pulse0` that contains only values for the first
pulse measurement `1 min` for all subjects. 

Create a new column  `pulse1` that contains only values for the last
pulse measurement `30 min` for all subjects.

Then, rescale `pulse` column **within subjects** using min-max
scaling.

Finally, write an expression operating on the transformed dataset that uses the `pulse`, `pulse0` and `pulse1`  columns
to restore the original values of the `pulse` column.

In [ ]:
df_correct = sbn.load_dataset('exercise')
df.head()

In [ ]:
df = sbn.load_dataset('exercise')

#Part I
df['pulse0'] = df['pulse'][::3].astype('Int64', errors='ignore')
df['pulse1'] = df['pulse'][2::3].astype('Int64', errors='ignore')
df
#Part II
df['pulse'] = df.groupby('id')['pulse'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
df
#Part III
df['pulse'] = df.groupby('id').apply(
    lambda x: x['pulse'] * (max(x['pulse1'].fillna(0)) - max(x['pulse0'].fillna(0))) + max(x['pulse0'].fillna(0))
    ).reset_index(level=0, drop=True)
df['pulse'] = round(df['pulse'])
df
#This solution doesn't transform pulse to its original form, because pulse in the 30th minute is not always the biggest one

In [ ]:
df = sbn.load_dataset('exercise')
df['pulse0'], df['pulse1'] = np.nan, np.nan
#Part I
for group, gdf in df.groupby('id')['pulse']:
  df.iloc[gdf.idxmin(), 6] = gdf.min()
  df.iloc[gdf.idxmax(), 7] = gdf.max()
df['pulse0'] = df['pulse0'].astype('Int64', errors='ignore')
df['pulse1'] = df['pulse1'].astype('Int64', errors='ignore')
df.head()
#Part II
df['pulse'] = df.groupby('id')['pulse'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
df
#Part III
df['pulse'] = df.groupby('id').apply(
    lambda x: x['pulse'] * (max(x['pulse1'].fillna(0)) - max(x['pulse0'].fillna(0))) + max(x['pulse0'].fillna(0))
    ).astype('int64').reset_index(level=0, drop=True)
df
df['pulse'].equals(df_correct['pulse'])
#This solution transforms pulse column into its original value

## Problem 8 

Draw a scatter plot matrix for `iris` dataset. It should show relationships between all pairs of numeric
variables and its diagonal should show histograms for individual variables. Points on scatter plots
and bars on histograms should be colored by `species`.

In [ ]:
iris = sbn.load_dataset('iris')
iris.head()

In [ ]:
sbn.PairGrid(iris, hue="species", palette=['mediumturquoise', 'crimson', 'gold']).map_diag(sbn.histplot).map_offdiag(sbn.scatterplot).add_legend(title="Iris spiecies:")


## Problem 9 

Use `titanic` dataset and draw bar plot showing distribution of `pclass` broken down by `embark_town`.

You can use [this example](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py)
as a guide for working with bar plots in Matplotlib.

In [ ]:
titanic = sbn.load_dataset('titanic')
titanic.head()


In [ ]:
class_town = titanic.groupby('pclass')['embark_town'].value_counts().to_dict()

labels = ['Southampton', 'Queenstown', 'Cherbourg']
one_class = [class_town[(1, 'Southampton')], class_town[(1, 'Queenstown')], class_town[(1, 'Cherbourg')]]
two_class = [class_town[(2, 'Southampton')], class_town[(2, 'Queenstown')], class_town[(2, 'Cherbourg')]]
three_class = [class_town[(3, 'Southampton')], class_town[(3, 'Queenstown')], class_town[(3, 'Cherbourg')]]

x = np.arange(len(labels)) 
width = 0.3

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, one_class, width, color='goldenrod', label='Class 1')
rects2 = ax.bar(x, two_class, width, color='cadetblue', label='Class 2')
rects3 = ax.bar(x + width, three_class, width, color='palevioletred', label='Class 3')

ax.set_ylabel('Number of passengers')
ax.set_xlabel('Embark town')
ax.set_title('Ditribution of class depending on embark town')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 0), 
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()

## Problem 13 

Look carefully at `exercise` dataset again. Remove junk column `Unnamed: 0` from it 
(this is already done in the loading chunk). 
Note that it is in the _long_ format.
Convert it to the _wide_ format. Which column allows you to identify properly all individual subjects? 
Think carefuly which columns are constant within a single subject. Values from non-constant columns should be spread out in separate columns.

After that, convert it back from the wide format to the long format, so it looks exactly like
in the beginning.

In [ ]:
df = sbn.load_dataset('exercise').loc[:, 'id':]
# This is technical issue beyond the scope of this course
# Do not remove this part
for col in ('time', 'diet', 'kind'):
    df[col] = df[col].astype(str)
    
df.head()

In [ ]:
#Long to wide
df_wide = df.pivot_table(index = ['id', 'diet', 'kind'], columns = 'time', values = 'pulse').reset_index()
df_wide

In [ ]:
#Wide to long
df_long = df_wide.melt(id_vars=['id', 'diet', 'kind'], value_name='pulse').sort_values(by='id').reset_index(drop=True)
df_long = df_long[['id','diet','pulse','time','kind']] #Setting order of columns

#Sorting values by time, because the order is changed
df_long = df_long.groupby('id').apply(lambda x: x.sort_values(by='time'))
#Setting indexes
df_long.reset_index(drop=True, inplace=True)
df_long
df_long.equals(df)

